# Demo visualisation

In [1]:
import numpy as np
import polygraphs as pg

from polygraphs import graphs
from polygraphs import hyperparameters as hparams
from polygraphs import visualisations as viz
from polygraphs import ops

%matplotlib inline

In [ ]:
# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.01

params.network.kind = 'cycle'
params.network.size = 8

# Watts Strogatz Parameters
params.network.wattsstrogatz.knn = 4
params.network.wattsstrogatz.probability = 0.5

# Barabasi-Albert Parameters
params.network.barabasialbert.attachments = 4

# Enable logging; print progress every 100 steps
params.logging.enabled = True
params.logging.interval = 100

# Take snapshots (incl. messages)
params.snapshots.enabled = True
params.snapshots.interval = 1
params.snapshots.messages = True

params.simulation.steps = 0
params.simulation.repeats = 10

# Store results in directory
params.simulation.results = "data/cycle"

# Set seed
#params.seed = 123456789

pg.random(params.seed)
_ = pg.simulate(params, op=ops.BalaGoyalOp)

In [2]:
from genericpath import exists
import os
from posixpath import splitext
import h5py
import json
import glob

import torch
import dgl
import networkx as nx


def __export_nodes(graph, sim):
    """
    Export nodes
    """
    filename = f"{sim}-nodes.json"

    data = []
    for nid in graph.nodes():
        belief = graph.ndata["beliefs"][nid].item()
        data.append({"id": nid.item(), "belief": belief})

    with open(filename, "w") as fstream:
        json.dump(data, fstream, indent=4)
    return


def __export_links(graph, sim):
    """
    Export links
    """
    filename = f"{sim}-links.json"

    data = []
    src, dst = graph.edges()
    for s, t in zip(src, dst):
        data.append({"source": s.item(), "target": t.item()})

    with open(filename, "w") as fstream:
        json.dump(data, fstream, indent=4)
    return


def __export_messages(graph, iid):
    """
    Export messages from a given iteration (`iid`).
    """
    data = []
    for u in graph.nodes():
        payoff = graph.ndata["payoffs"][u].item()
        sample = graph.ndata["samples"][u].item()

        # Ingore "empty" messages
        if sample < 1:
            continue

        # Iterate over node's neighbours
        neighbours = graph.out_edges(u, form="uv")[1]
        for v in neighbours:
            data.append(
                {
                    "iid": iid,
                    "source": u.item(),
                    "target": v.item(),
                    "payoff": payoff,
                    "sample": sample,
                }
            )

    # Return messages from given iteration
    return data


def __export_beliefs(graph, iid):
    """
    Export beliefs from a given iteration (`iid`).
    """
    data = []
    for nid in graph.nodes():
        belief = graph.ndata["beliefs"][nid].item()
        data.append({"iid": iid, "id": nid.item(), "belief": belief})
    return data


def __export_iterations(graph, snapshots, sim):
    """
    Export iterations
    """
    # Output file
    filename = f"{sim}-iterations.json"

    _keys = [int(key) for key in snapshots["beliefs"].keys()]
    _keys = sorted(_keys)

    data = []
    for key in _keys:
        # Populate graph node attributes
        graph.ndata["beliefs"] = torch.tensor(snapshots["beliefs"][str(key)][:])
        graph.ndata["payoffs"] = torch.tensor(snapshots["payoffs"][str(key)][:].T[0])
        graph.ndata["samples"] = torch.tensor(snapshots["payoffs"][str(key)][:].T[1])

        # Export messages
        data += __export_messages(graph, key)

        # Export new node beliefs
        data += __export_beliefs(graph, key)

    with open(filename, "w") as fstream:
        json.dump(data, fstream, indent=4)


def __export_groupbeliefs(graph, snapshots, sim):
    def majority(beliefs, threshold=0.5, weights=None):
        """
        Returns percentage of nodes that believe B is better.
        """
        if weights is None:
            weights = torch.ones(beliefs.shape)

        zeros = torch.zeros(beliefs.shape, dtype=weights.dtype)

        # Count (normalized) votes
        votes = torch.where(beliefs > threshold, weights, zeros)
        result = votes.sum() / weights.sum()
        return result.item()

    # Output file
    filename = f"{sim}-groupbeliefs.json"

    _keys = [int(key) for key in snapshots["beliefs"].keys()]
    _keys = sorted(_keys)

    data = []

    for key in _keys:
        beliefs = torch.tensor(snapshots["beliefs"][str(key)][:])
        group_beliefs = {"iid": key}

        # Is there a simple majority (> 0.5) of NODES whose credence exceeds the threshold (>0.99)?
        group_beliefs["belief_unweighted"] = majority(
            beliefs=beliefs, threshold=0.99, weights=None
        )

        group_beliefs["belief_weighted"] = majority(
            beliefs=beliefs, threshold=0.99, weights=graph.in_degrees()
        )

        group_beliefs["credence_unweighted"] = str(np.average(a=beliefs))
        group_beliefs["credence_weighted"] = str(
            np.average(a=beliefs, weights=graph.in_degrees())
        )

        data.append(group_beliefs)

    with open(filename, "w") as fstream:
        json.dump(data, fstream, indent=4)


def toJSON(sim):
    """
    Post-process graph snapshots and export them into JSON format
    """
    # Load DGL graph
    graphs, _ = dgl.load_graphs(f"{sim}.bin")
    graph = graphs[0]

    # Remove self-loops
    graph = dgl.remove_self_loop(graph)

    # Export nodes
    __export_nodes(graph, sim)

    # Export links
    __export_links(graph, sim)

    # Read snapshots
    snapshots = h5py.File(f"{sim}.hd5", "r")

    # Export iterations
    __export_iterations(graph, snapshots, sim)

    # Export group beliefs
    __export_groupbeliefs(graph, snapshots, sim)

    return


def export(directory, sims):
    # Ensure that the directory exists
    assert os.path.isdir(directory)
    # If sims is None, find number of simulations
    if sims is None:
        # List all binary graph files
        sims = glob.glob(os.path.join(directory, "*.bin"))
        # Ignore file extensions
        sims = [os.path.splitext(filename)[0] for filename in sims]
    else:
        # Process a user-specified list
        if not isinstance(sims, list):
            sims = [sims]
        # Ensure all inputs are strings
        assert all(isinstance(x, str) for x in sims)
        # Ensure all inputs result in a valid simulation file
        for sid in sims:
            filename = os.path.join(directory, f"{sid}.bin")
            assert os.path.exists(filename), f"File not found: {filename}"
    # Processing simulations
    for sim in sims:
        toJSON(sim)


# Main
directory = "data/cycle/"
sims = None

export(directory, sims)
print("Bye.")

Bye.
